# OpenSea V1


In [2]:
import pandas as pd

from credmark.cmf.ipython import create_cmf
from credmark.cmf.types import Token, Contract, Address, Account, BlockNumber, Records

# fix_block_number = None
fix_block_number = 17527032

cmf_param = {
    'block_number': fix_block_number,
    'chain_id': 1,
    'use_local_models': '*',
}
context, _model_loader = create_cmf(cmf_param, True)
fix_block_number = context.block_number
print(fix_block_number)

Credmark context created with 
- chain_id=1
- block_number=17527032
- chain_to_provider_url=http://192...8545
- model_loader_path=['/home/yangye/dev/credmark/credmark-models-py/models']
- api_url=https://gateway.credmark.com
- use_local_models=*

17527032


In [4]:
opensea_v1 = Contract('0x7Be8076f4EA4A4AD08075C2508e481d6C946D12b')

opensea_v1_blk = context.run_model('token.deployment', opensea_v1)[  # type: ignore
    'deployed_block_number']
opensea_v1_blk

5774644

In [30]:
df = pd.DataFrame(opensea_v1.fetch_events(
    opensea_v1.events.OrdersMatched,
    from_block=5774644,
    to_block=5774644+10000,
    contract_address=opensea_v1.address))

if not df.empty:
    df = (
        df.loc[:, ['blockNumber', 'logIndex', 'maker', 'taker',
                   'metadata', 'buyHash', 'sellHash', 'price', 'transactionHash']]
        .assign(metadata=lambda x: x.metadata.apply(lambda y: y.hex()),
                buyHash=lambda x: x.buyHash.apply(lambda y: y.hex()),
                sellHash=lambda x: x.sellHash.apply(lambda y: y.hex()),
                transactionHash=lambda x: x.transactionHash.apply(lambda y: y.hex()),
                price=lambda x: x.price / 1e18,)
    )

    display(df.loc[0].T)

    display(df.loc[0].metadata)

    display(len(df.loc[0].transactionHash),
            len(df.loc[0].metadata),
            len(df.loc[0].buyHash),
            len(df.loc[0].sellHash),
            )

# df.loc[:, 'txHash'] = df.transactionHash.apply(lambda x: x.hex())
# df.query('txHash == "0x7efdbef1c377126b5bb5381b8f46a5dd7cb4b992b5f2f48a5f47eec86297c281"').T

blockNumber                                                  5779378
logIndex                                                          55
maker                     0x6Be4a7bbb812Bfa6A63126EE7b76C8A13529BDB8
taker                     0x0239769A1aDF4DeF9f07Da824B80B9C4fCB59593
metadata           0000000000000000000000000000000000000000000000...
buyHash            c1f12102332b8d0dfe98a6e2045860dfd22a30388c97b4...
sellHash           0000000000000000000000000000000000000000000000...
price                                                          0.001
transactionHash    0xb066452c1e87696392c9e74fd0159dea8b53d2162138...
Name: 0, dtype: object

'0000000000000000000000000000000000000000000000000000000000000000'

66

64

64

64

In [ ]:
df = pd.DataFrame(opensea_v1.fetch_events(
    opensea_v1.events.Orders,
    from_block=14029202,
    to_block=14029202,
    contract_address=opensea_v1.address))